In [2]:
import warnings
warnings.filterwarnings('ignore')
import copy
import pandas as pd
import numpy as np
import random

import requests
from bs4 import BeautifulSoup

import time
import random
import re
import unicodedata

from sportsreference.ncaab.boxscore import Boxscore
from sportsreference.ncaab.roster import Player
from sklearn.ensemble import GradientBoostingClassifier
import sportsreference.nba.roster as NBA

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
#use beautiful soup to get players from 1985-2016 Nba overall stats
lst1 = []
lst2 = []
#use a count to keep track of whats going on
count = 0
draft_url = 'https://www.basketball-reference.com/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2016&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=0'
for i in range(45):
    if i >0:
        href_list = soup.find("div", {"class": "p402_premium"}).find('p').find_all('a')
        if len(href_list) == 1:
            count += 1
            #print count to see whats going on
            print(count)
            #I don't want it to be an infinite loop so when it hits the last page it will break
            if count == 2:
                break
            href = href_list[0].get('href')
        else:
            href = href_list[1].get('href')
        print(href)
        draft_url= 'https://www.basketball-reference.com' + href
        time.sleep(3*random.random())
    r = requests.get(draft_url)
    #200 status code means it worked
    print(r.status_code)
    soup = BeautifulSoup(r.content, "html")
    div = soup.find("div", {"class": "table_outer_container"})
    table = div.find("tbody")
    rows = table.find_all('tr')
    for row in rows:
        try:
            href = row.find_all('a')[1].get('href')
        except:
            continue
        try:
            #using regex I find the nba ID to be used later
            nba_id = re.findall('\w+\d{2}', href)[0]
        except:
            #if they don't have an nba id skip them because that means they never played
            continue
        tds = row.find_all('td')
        lst3 = []
        #get all stats
        for item in tds:
            lst3.append(item.get_text())
        lst3.append(nba_id)
        lst1.append(lst3)
#name of the kind of stat
for i in rows[0].find_all('td'):
    lst2.append(i.get("data-stat"))
        

200
1
/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2016&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=100
200
/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2016&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=&c1comp=&c1val=&c2stat=&c2comp=&c2val=&c3stat=&c3comp=&c3val=&c4stat=&c4comp=&c4val=&order_by=ws&order_by_asc=&offset=200
200
/play-index/draft_finder.cgi?request=1&year_min=1985&year_max=2016&round_min=&round_max=&pick_overall_min=&pick_overall_max=&franch_id=&college_id=0&is_active=&is_hof=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_

In [4]:
random.shuffle(lst1)
print(len(lst1))

1720


In [5]:
#function to strip accents

def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)

In [6]:
def correct_college_id(element):
    
    player = strip_accents_and_punc(element[5])
    split_player = player.split()
    #try all jrs first
    college_id = ('-'.join(split_player) + '-jr-1').lower()

    draft_year = int(element[0][-2:])

    injuried_year = int(draft_year - 1)
    
    try:
        #if the last college year is not the same a nba draft year assume they had an injury and return id
        last_college_year = int(Player(college_id).dataframe.index[-2][0].split('-')[-1])
        if ((last_college_year == draft_year) | (last_college_year == injuried_year)):
            return college_id
    except:
        #if the years don't match or aren't off by 1 year, its probably a different player with the same name
        pass
    #now trying names without jr
    for attempt in range(1,6):
        split_player = player.split()
        #changing the player id to be james-harden-1, james-harden-2 up to 6 in order to find correct id
        college_id = ('-'.join(split_player) + '-' + str(attempt)).lower()
        try:
            last_college_year = int(Player(college_id).dataframe.index[-2][0].split('-')[-1])
            #if the last college year is not the same a nba draft year assume they had an injury and return id 
            if ((last_college_year == draft_year) | (last_college_year == injuried_year)):
                return college_id
            continue
        except:
            continue
        #if no college id returns then they didn't go to college
        return 'not_college_player'

In [7]:
#gets rid of duplicates paricutlary useful for Patrick ewing and Patrick ewing Jr

def get_rid_of_dup_names(name,college_id,df):
        name = strip_accents(name)
        id_split = college_id.split('-')
        if len(id_split) > 2:
            if id_split[-2] == 'jr':
                return name + ' Jr'
        if name in df.index:
            return name + ' ' + id_split[-1]
        return name

In [8]:
def strip_accents_and_punc(name):
    name = strip_accents(name).replace('.','').replace("'",'').replace(',','')
    return name

In [9]:
#get all the allstars
all_star_list = []
url = 'https://www.basketball-reference.com/allstar/NBA-allstar-career-stats.html'
r = requests.get(url)
soup1 = BeautifulSoup(r.content, "html")
div = soup1.find("div", {"class": "table_outer_container"})
table = div.find("tbody")
rows = table.find_all('tr')
for row in rows:
    a_tag =row.find('th').find('a')
    #if theres nothing there, skip it
    if a_tag == None:
        continue
    #get the all-star players name
    player = a_tag.get_text()
    #get a link with the all-stars player id
    href = a_tag.get('href')
    #use regex to find the id
    nba_id = re.findall('\w+\d{2}', href)[0]
    #add the player to the list
    all_star_list.append(nba_id)

In [10]:
#functions rename columns
def rename_nba_index(nba_player_series):
    NBA_index = nba_player_series.index
    new_index = []
    for index in NBA_index:
        new_index.append(index+'_NBA')
    nba_player_series.index = new_index
    return nba_player_series
def rename_college_index(college_player_series):
    college_index = college_player_series.index
    new_index = []
    for index in college_index:
        new_index.append(index+'_COLLEGE')
    college_player_series.index = new_index
    return college_player_series

In [11]:
#puts two series together
def combine_nba_college(college_series,nba_series):
    return pd.concat([college_series,nba_series])

In [20]:
#creating a dataframe to use in analysis
data = pd.DataFrame()

for playr in lst1:
    player = strip_accents_and_punc(playr[5])
    college_id = correct_college_id(playr)
    nba_id = playr[-1]

    player = get_rid_of_dup_names(player,college_id,data)

    try:
        college_dataframe = Player(college_id).dataframe
        college_series = college_dataframe.iloc[-1,:]
        nba_series = NBA.Player(nba_id).dataframe.iloc[0,:]
    except:
        continue
    #create rows in dataframe for each player
    college_series['conference'] = college_dataframe.iloc[-2,:]['conference']
    college_series['years_in_college'] = len(college_dataframe.index) - 1
    college_series['nba_position'] = playr[7]
    college_series.drop(labels=['player_id'])
    nba_series['nba_id'] = nba_id
    nba_series = rename_nba_index(nba_series)
    college_series = rename_college_index(college_series)
    combined = combine_nba_college(college_series,nba_series)
    data = pd.concat([data, pd.DataFrame(combined.values.reshape(1,len(combined.values)),
                                         index=[player],
                                         columns=[combined.index])])
data

1
not_college_player
Juan Pablo Vaulet
2
klay-thompson-1
Klay Thompson
3
anthony-bennett-1
Anthony Bennett
4
cedric-henderson-2
Cedric Henderson
5
patrick-mccaw-1
Patrick McCaw
6
not_college_player
Olumide Oyedeji
7
leon-powe-1
Leon Powe
8
tyler-honeycutt-1
Tyler Honeycutt
9
not_college_player
Arturas Gudaitis
10
jabari-smith-1
Jabari Smith
11
alex-len-1
Alex Len
12
daequan-cook-1
Daequan Cook
13
calvin-booth-1
Calvin Booth
14
reggie-miller-1
Reggie Miller
15
maarty-leunen-1
Maarty Leunen
16
jimmy-king-1
Jimmy King
17
mark-acres-1
Mark Acres
18
clarence-weatherspoon-1
Clarence Weatherspoon
19
torraye-braggs-1
Torraye Braggs
20
aaron-gordon-1
Aaron Gordon
21
not_college_player
Vlade Divac
22
roy-tarpley-1
Roy Tarpley
23
rob-lock-1
Rob Lock
24
rodney-monroe-1
Rodney Monroe
25
noah-vonleh-1
Noah Vonleh
26
not_college_player
Bruno Sundov
27
olivier-hanlan-1
Olivier Hanlan
28
adonal-foyle-1
Adonal Foyle
29
cody-zeller-1
Cody Zeller
30
todd-lichti-1
Todd Lichti
31
dikembe-mutombo-1
Dikembe M

245
kirk-hinrich-1
Kirk Hinrich
246
royce-white-1
Royce White
247
dana-barros-1
Dana Barros
248
gary-trent-1
Gary Trent
249
dallas-comegys-1
Dallas Comegys
250
acie-earl-1
Acie Earl
251
stephane-lasme-1
Stephane Lasme
252
cory-joseph-1
Cory Joseph
253
trevor-ariza-1
Trevor Ariza
254
bobby-simmons-1
Bobby Simmons
255
not_college_player
Beno Udrih
256
jared-sullinger-1
Jared Sullinger
257
daniel-hamilton-1
Daniel Hamilton
258
randy-brown-1
Randy Brown
259
david-lee-1
David Lee
260
carl-landry-1
Carl Landry
261
not_college_player
Adam Hanga
262
rumeal-robinson-1
Rumeal Robinson
263
jamal-mashburn-1
Jamal Mashburn
264
ater-majok-1
Ater Majok
265
not_college_player
Zydrunas Ilgauskas
266
dave-jamerson-1
Dave Jamerson
267
not_college_player
Stefano Rusconi
268
solomon-jones-1
Solomon Jones
269
armen-gilliam-1
Armen Gilliam
270
byron-irvin-1
Byron Irvin
271
jon-brockman-1
Jon Brockman
272
not_college_player
Vassilis Spanoulis
273
jerome-james-1
Jerome James
274
luc-longley-1
Luc Longley
275
s

486
andrew-lang-1
Andrew Lang
487
terry-catledge-1
Terry Catledge
488
al-horford-1
Al Horford
489
joel-bolomboy-1
Joel Bolomboy
490
kenny-satterfield-1
Kenny Satterfield
491
jalen-rose-1
Jalen Rose
492
derrick-chievous-1
Derrick Chievous
493
von-wafer-1
Von Wafer
494
larry-hughes-1
Larry Hughes
495
antonio-lang-1
Antonio Lang
496
jason-sasser-1
Jason Sasser
497
reece-gaines-1
Reece Gaines
498
tyus-jones-1
Tyus Jones
499
walter-palmer-1
Walter Palmer
500
kevin-edwards-1
Kevin Edwards
501
bj-armstrong-1
BJ Armstrong
502
donnell-harvey-1
Donnell Harvey
503
anthony-miller-1
Anthony Miller
504
not_college_player
Furkan Aldemir
505
joey-graham-1
Joey Graham
506
robert-whaley-1
Robert Whaley
507
not_college_player
Dino Raa
508
shawne-williams-1
Shawne Williams
509
not_college_player
Danilo Gallinari
510
doug-smith-1
Doug Smith
511
rodrick-rhodes-1
Rodrick Rhodes
512
trenton-hassell-1
Trenton Hassell
513
greg-foster-1
Greg Foster
514
pat-durham-1
Pat Durham
515
jahidi-white-1
Jahidi White
516


727
derrick-favors-1
Derrick Favors
728
william-avery-1
William Avery
729
ike-diogu-1
Ike Diogu
730
grant-long-1
Grant Long
731
shane-battier-1
Shane Battier
732
travis-best-1
Travis Best
733
not_college_player
Albert Miralles
734
michael-gbinije-1
Michael Gbinije
735
steve-alford-1
Steve Alford
736
not_college_player
Willy Hernangomez
737
jason-collier-1
Jason Collier
738
jrue-holiday-1
Jrue Holiday
739
deandre-bembry-1
DeAndre Bembry
740
michael-adams-1
Michael Adams
741
george-hill-1
George Hill
742
david-vaughn-1
David Vaughn
743
jordan-mickey-1
Jordan Mickey
744
ron-grandison-1
Ron Grandison
745
bryon-russell-1
Bryon Russell
746
eric-riley-1
Eric Riley
747
jason-hart-1
Jason Hart
748
eddie-house-1
Eddie House
749
not_college_player
Andris Biedrins
750
not_college_player
Dirk Nowitzki
751
not_college_player
Bostjan Nachbar
752
jason-kidd-1
Jason Kidd
753
felipe-lopez-1
Felipe Lopez
754
not_college_player
Anderson Varejao
755
roy-rogers-1
Roy Rogers
756
andre-moore-1
Andre Moore
757

969
grant-hill-1
Grant Hill
970
bison-dele-1
Bison Dele
971
michael-anderson-1
Michael Anderson
972
darrin-hancock-1
Darrin Hancock
973
nerlens-noel-1
Nerlens Noel
974
not_college_player
Bruno Caboclo
975
not_college_player
Livio Jean-Charles
976
not_college_player
Marko Todorovic
977
julius-hodge-1
Julius Hodge
978
chris-mihm-1
Chris Mihm
979
ed-stokes-1
Ed Stokes
980
litterial-green-1
Litterial Green
981
todd-macculloch-1
Todd MacCulloch
982
matt-bonner-1
Matt Bonner
983
marquis-teague-1
Marquis Teague
984
brian-shaw-1
Brian Shaw
985
jackson-vroman-1
Jackson Vroman
986
ron-mercer-1
Ron Mercer
987
jeff-trepagnier-1
Jeff Trepagnier
988
sam-cassell-1
Sam Cassell
989
not_college_player
John Turner
990
jonny-flynn-1
Jonny Flynn
991
not_college_player
Ejike Ugboaja
992
nikita-wilson-1
Nikita Wilson
993
isaiah-whitehead-1
Isaiah Whitehead
994
not_college_player
Kebu Stewart
995
eric-maynor-1
Eric Maynor
996
not_college_player
Efthimi Rentzias
997
milos-babic-1
Milos Babic
998
linas-kleiza-1

1203
not_college_player
Arvydas Sabonis
1204
shelvin-mack-1
Shelvin Mack
1205
darren-collison-1
Darren Collison
1206
joakim-noah-1
Joakim Noah
1207
not_college_player
Nikoloz Tskitishvili
1208
not_college_player
Marko Milic
1209
hamady-ndiaye-1
Hamady NDiaye
1210
scott-burrell-1
Scott Burrell
1211
kendall-gill-1
Kendall Gill
1212
george-lynch-1
George Lynch
1213
tyrone-hill-1
Tyrone Hill
1214
not_college_player
Dorell Wright
1215
jarron-collins-1
Jarron Collins
1216
dangelo-russell-1
DAngelo Russell
1217
richard-hamilton-1
Richard Hamilton
1218
mike-taylor-1
Mike Taylor
1219
kosta-koufos-1
Kosta Koufos
1220
stacey-king-1
Stacey King
1221
not_college_player
Yaroslav Korolev
1222
daniel-ewing-1
Daniel Ewing
1223
malik-beasley-1
Malik Beasley
1224
pat-connaughton-1
Pat Connaughton
1225
ed-obannon-1
Ed OBannon
1226
mike-dunleavy-2
Mike Dunleavy
1227
delonte-west-1
Delonte West
1228
not_college_player
DeShawn Stevenson
1229
scot-pollard-1
Scot Pollard
1230
tim-hardaway-jr-1
Tim Hardaway Jr


1435
not_college_player
Jeremy Tyler
1436
jared-cunningham-2
Jared Cunningham
1437
marshon-brooks-1
MarShon Brooks
1438
not_college_player
Patrick Ewing not_college_player
1439
arsalan-kazemi-1
Arsalan Kazemi
1440
tim-thomas-1
Tim Thomas
1441
blair-rasmussen-1
Blair Rasmussen
1442
rashad-mccants-1
Rashad McCants
1443
jared-dudley-1
Jared Dudley
1444
not_college_player
Venson Hamilton
1445
francisco-elson-1
Francisco Elson
1446
not_college_player
Manute Bol
1447
mardy-collins-1
Mardy Collins
1448
semaj-christon-1
Semaj Christon
1449
shawn-marion-1
Shawn Marion
1450
not_college_player
Charles Oakley
1451
ryan-anderson-1
Ryan Anderson
1452
not_college_player
Nikola Radicevic
1453
not_college_player
Alexis Ajinca
1454
kenny-green-1
Kenny Green
1455
kawhi-leonard-1
Kawhi Leonard
1456
brian-grant-1
Brian Grant
1457
not_college_player
Wang Zhizhi
1458
not_college_player
Arvydas Sabonis
1459
deandre-daniels-1
DeAndre Daniels
1460
sam-vincent-1
Sam Vincent
1461
andre-turner-1
Andre Turner
1462


1667
josh-boone-1
Josh Boone
1668
kevin-love-1
Kevin Love
1669
moochie-norris-1
Moochie Norris
1670
jerry-reynolds-1
Jerry Reynolds
1671
not_college_player
Nicolas Batum
1672
dion-glover-1
Dion Glover
1673
greg-anthony-1
Greg Anthony
1674
antonio-davis-1
Antonio Davis
1675
ronnie-murphy-1
Ronnie Murphy
1676
not_college_player
David Andersen
1677
omar-cook-1
Omar Cook
1678
bo-kimble-1
Bo Kimble
1679
greg-oden-1
Greg Oden
1680
not_college_player
Ricky Rubio
1681
buck-johnson-1
Buck Johnson
1682
jon-leuer-1
Jon Leuer
1683
chris-welp-1
Chris Welp
1684
not_college_player
Bojan Bogdanovic
1685
will-blalock-1
Will Blalock
1686
bobby-hurley-1
Bobby Hurley
1687
jajuan-johnson-1
JaJuan Johnson
1688
not_college_player
Luc Mbah a Moute
1689
anthony-parker-1
Anthony Parker
1690
robert-sacre-1
Robert Sacre
1691
derrick-brown-1
Derrick Brown
1692
tony-snell-1
Tony Snell
1693
joe-johnson-1
Joe Johnson
1694
john-thomas-1
John Thomas
1695
darnell-jackson-1
Darnell Jackson
1696
gerald-paddio-1
Gerald Pad

,assist_percentage_COLLEGE,assists_COLLEGE,block_percentage_COLLEGE,blocks_COLLEGE,box_plus_minus_COLLEGE,conference_COLLEGE,defensive_box_plus_minus_COLLEGE,defensive_rebound_percentage_COLLEGE,defensive_rebounds_COLLEGE,defensive_win_shares_COLLEGE,...,two_point_attempts_NBA,two_point_percentage_NBA,two_pointers_NBA,two_pointers_assisted_percentage_NBA,usage_percentage_NBA,value_over_replacement_player_NBA,weight_NBA,win_shares_NBA,win_shares_per_48_minutes_NBA,nba_id_NBA
Klay Thompson,20.3,259,2.7,74,10.1,pac-10,3.2,13.8,386,5.5,...,450,0.46,207,0.623,24.7,0.1,215,1.7,0.05,thompkl01
Anthony Bennett,8.8,34,4.6,43,8.5,mwc,3.2,24,199,2.2,...,172,0.39,67,0.612,20.3,-0.9,245,-0.4,-0.028,bennean01
Cedric Henderson,None,181,None,114,None,cusa,None,None,None,4.6,...,721,0.483,348,NaN,17.8,1,215,3.8,0.072,hendece02
Patrick McCaw,20.7,214,1.1,23,6.3,mwc,2.3,10.8,212,3.7,...,122,0.533,65,0.723,11.9,0.1,185,1.7,0.077,mccawpa01
Leon Powe,8.2,57,1.4,32,None,pac-10,None,None,348,3.8,...,175,0.451,79,0.354,18.1,-0.4,240,1.7,0.116,powele01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mike Muscala,13.7,211,7.9,271,8.4,patriot,4.6,24.9,792,8.3,...,70,0.443,31,0.839,19.3,-0.2,240,0.2,0.04,muscami01
Jordan Clarkson,20.5,249,1,26,3,sec,0,9.4,236,3.2,...,475,0.482,229,0.279,23,0.6,194,2.4,0.078,clarkjo01
Dominic McGuire,10.5,172,4.1,167,None,wac,None,None,392,6.8,...,97,0.392,38,0.658,10.4,-0.2,220,0,-0.003,mcguido01
Latrell Sprewell,None,136,None,85,None,sec,None,None,None,None,...,770,0.488,376,NaN,19.3,1.7,190,4,0.07,sprewla01


In [21]:
#columns have unwanted items so get rid of those.
data.columns = [col[0] for col in data.columns]

In [40]:
#save dataframe

#data.to_pickle('data.pkl')

In [13]:
data = pd.read_pickle('data.pkl')

In [22]:
#find all the ids of all stars
nba_id_list = []
for player_id in data['player_id_NBA'].values:
    nba_id_list.append(player_id)

In [23]:
#adding a target column to see whether they are an all-star or not
all_star_col = []
for player in nba_id_list:
    for all_star in all_star_list:
        if all_star == player:
            all_star_col.append(1)
            break
        if all_star == all_star_list[-1]:
            all_star_col.append(0)

In [24]:
y = pd.DataFrame(np.array(all_star_col),index=data.index,columns=['all_star?'])

In [36]:
#function gets the NBA PER
def get_career_per(dataframe):
    career_per_list = []
    temp = dataframe['player_id_NBA']
    r = 0
    for player in temp.values:
        r +=1
        print(r)
        try: 
            career_per = NBA.Player(player).dataframe.loc['Career','player_efficiency_rating']
        except:
            #did this only to find errors
            return (player, r)
        career_per_list.append(career_per)
    return pd.DataFrame(np.array(career_per_list),index=dataframe.index,columns=['career_per'])

In [38]:
career_per = get_career_per(data)
career_per


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,career_per
Klay Thompson,16.4
Anthony Bennett,10.2
Cedric Henderson,9.2
Patrick McCaw,7.9
Leon Powe,16.8
...,...
Mike Muscala,13.6
Jordan Clarkson,15.3
Dominic McGuire,9.8
Latrell Sprewell,15.1


In [48]:
#drop duplicates and ids and others with strings that are irrelevant
data.drop(columns =['nba_id_NBA','height_NBA','player_id_COLLEGE','player_id_NBA','nationality_NBA','position_NBA','team_abbreviation_NBA'], inplace=True)

In [49]:
#add dummies columns
data = pd.get_dummies(data=data,columns=['position_COLLEGE','nba_position_COLLEGE','conference_COLLEGE','team_abbreviation_COLLEGE']) 

In [50]:
#function returns height in feet as a float
def height_func(row):
    try: 
        split = row['height_COLLEGE'].split('-')
    except:
        return -1
    return int(split[0])+(int(split[1])/12)

In [51]:
data['height_COLLEGE'] = data.apply(height_func ,axis=1)

In [52]:
for column in data.columns:
    data[column].astype(float)
    length_of_nulls=len(data[data[column].isnull()])
    if length_of_nulls > 0:
        data[column +' is missing'] = data[column].isnull()*1
data.fillna(-1, inplace=True)

In [59]:
train_data, test_data, y_train, y_test = train_test_split(data, y)

In [61]:
grad_boost = GradientBoostingClassifier(learning_rate=.01, n_estimators=5000)

In [62]:
grad_boost.fit(train_data, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=5000,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [63]:
yhat = grad_boost.predict(test_data)
tn, fp, fn, tp = confusion_matrix(y_test,yhat).ravel()
print((tn, fp, fn, tp))
print(roc_auc_score(y_test.values.flatten(),yhat))
print(test_data.index[((yhat==1) & (y_test.values.flatten())==1)])
print(test_data.index[((yhat==0) & (y_test.values.flatten())==1)])
print(test_data.index[((yhat==1) & (y_test.values.flatten())==0)])
print(type(y_test))

(291, 13, 30, 8)
0.583881578947
Index(['Carmelo Anthony', 'Steve Francis', 'Larry Johnson', 'James Harden',
       'Derrick Coleman', 'Jason Kidd', 'Elton Brand', 'Tim Duncan'],
      dtype='object')
Index(['Reggie Lewis', 'Chris Gatling', 'Antonio McDyess', 'Brook Lopez',
       'Klay Thompson', 'Joakim Noah', 'Karl Malone', 'Paul George',
       'Jeff Teague', 'Dan Majerle', 'Dana Barros', 'Joe Johnson',
       'Chris Kaman', 'Jimmy Butler', 'Paul Millsap', 'A.C. Green',
       'Kevin Duckworth', 'Draymond Green', 'Jamal Mashburn', 'Patrick Ewing',
       'Andre Drummond', 'Terrell Brandon', 'Danny Manning', 'Jamaal Magloire',
       'Brad Daugherty', 'Glen Rice', 'Danny Granger', 'DeMarcus Cousins',
       'Latrell Sprewell', 'Rajon Rondo'],
      dtype='object')
Index(['Antonio Lang', 'Brandon Bass', 'Perry Jones', 'Reggie Lewis',
       'Alec Burks', 'Don Reid', 'Jordan Williams', 'Marcus Haislip',
       'Darius Morris', 'Eric Montross',
       ...
       'Tony Dumas', 'Will Blal

In [54]:
def add_all_star(dataframe):
    dataframe['all_star?'] = all_star_col
    return dataframe

In [55]:
def shuffle_df(dataframe):
    dataframe = shuffle(dataframe)
    return dataframe

In [66]:
def get_y(dataframe):
    return pd.DataFrame(dataframe['all_star?'].values,index=dataframe.index,columns=['all_star?'])

In [45]:
def get_y_per(dataframe):
    return pd.DataFrame(dataframe['career_per'].values,index=dataframe.index,columns=['career_per'])

In [46]:
def drop_y(dataframe):
    dataframe.drop(columns=['all_star?'],inplace=True)
    return dataframe

In [47]:
def drop_y_per(dataframe):
    dataframe.drop(columns=['career_per'],inplace=True)
    return dataframe

In [71]:
fold_scores = []
ct = 0
for i in range(30):
    dataframe = data.copy()
    temp = add_all_star(dataframe)
    shuffled = shuffle_df(temp)
    y2 = get_y(shuffled)
    dataframe = drop_y(shuffled)
    train_data1, test_data1, y_train1, y_test1 = train_test_split(dataframe, y2,random_state = int(100*random.random()))
    model = GradientBoostingClassifier(learning_rate=.01, n_estimators=5000)
    model.fit(train_data1, y_train1)
    yhat2 = model.predict(test_data1)
    auc = roc_auc_score(y_test1.values.flatten(),yhat2)
    print(auc)
    ct += 1
    print(ct)
    fold_scores.append(auc)
print('AVG: ', np.mean(fold_scores))
#AVG:  0.636415606783

0.619467994089
0.589321192053
0.628289473684
0.682406702209
0.622020381037
0.681531703591
0.621800529568
0.736217948718
0.667156286722
0.605339923831
0.591012947449
0.607258064516
0.622873825844
0.658817165525
0.624495967742
0.609298928919
0.691085613416
0.642284186402
0.764801375095
0.652724856004
0.615003808073
0.605263157895
0.621710526316
0.606649014416
0.638035815574
0.608552631579
0.60288506282
0.608403148007
0.634044173648
0.633715798764
AVG:  0.636415606783


In [27]:
probs = grad_boost.predict_proba(test_data)
prob_list = []
for element in probs:
    prob_list.append(element[1])
prob_array = np.array(prob_list)
index_array = np.array(test_data.index)
print(index_array[prob_array>.4])
#grad_boost.predict(data.loc['Frank Kaminsky'])

['Anthony Davis' 'Chris Paul' 'Paul Pierce' 'Nick Vanos' 'James Harden'
 'Stephen Curry' 'Brad Daugherty' 'Carlos Boozer' 'Anfernee Hardaway'
 'Dikembe Mutombo' 'Larry Johnson' 'Dan Majerle' 'Andrew Bogut'
 'Michael Carter-Williams' 'Karl-Anthony Towns' 'Andre Drummond'
 'John Williams' 'Christian Laettner' 'John Salley' 'Stephon Marbury'
 'James Posey' 'Kevin Durant' 'Willie Anderson' 'Jerry Stackhouse'
 'Brandon Roy' 'Harold Keeling']


In [25]:
random_forest=RandomForestClassifier()

In [26]:
random_forest.fit(train_data, nba)
yhat = random_forest.predict(test_data)
tn, fp, fn, tp = confusion_matrix(nba2,yhat).ravel()
print((tn, fp, fn, tp))
print(test_data.index[((np.array(yhat)==1) & (np.array(nba2)==1))])
print(roc_auc_score(nba2,yhat))
print(test_data.index[((np.array(yhat)==1) & (np.array(nba2)==0))])
print(test_data.index[((np.array(yhat)==0) & (np.array(nba2)==1))])

(285, 6, 39, 10)
Index(['Rik Smits', 'Anthony Davis', 'Kevin Love', 'Damian Lillard',
       'Brad Daugherty', 'Dikembe Mutombo', 'Larry Johnson',
       'Christian Laettner', 'Stephon Marbury', 'Jerry Stackhouse'],
      dtype='object')
0.591731537976
Index(['Rex Chapman', 'Brent Barry', 'Emeka Okafor', 'Andrew Bogut',
       'John Salley', 'Willie Anderson'],
      dtype='object')
Index(['Mookie Blaylock', 'Jamaal Magloire', 'Roy Hibbert', 'Chris Paul',
       'Paul Pierce', 'Nikola Vučević', 'B.J. Armstrong', 'Kenny Anderson',
       'Gilbert Arenas', 'Sam Cassell', 'David Lee', 'James Harden',
       'Jrue Holiday', 'Larry Nance', 'D'Angelo Russell', 'Stephen Curry',
       'Dana Barros', 'Rajon Rondo', 'David West', 'Carlos Boozer',
       'Anfernee Hardaway', 'Glenn Robinson', 'Dan Majerle', 'Gordon Hayward',
       'Allan Houston', 'Danny Manning', 'Karl-Anthony Towns',
       'Michael Finley', 'Andre Drummond', 'Glen Rice', 'Hersey Hawkins',
       'Chris Gatling', 'Jeff Teague

def height_func2(row):
    try: 
        split = row['height'].split('-')
    except:
        return 6.56
    return int(split[0])+(int(split[1])/12)

In [86]:
new_data = data.copy()
new_data['all_star?'] = y.values

#nba_position_COLLEGE_C-F
#nba_position_COLLEGE_F
#nba_position_COLLEGE_F-C
#nba_position_COLLEGE_F-G
#nba_position_COLLEGE_G
#nba_position_COLLEGE_G-F
centers = new_data[new_data['nba_position_COLLEGE_C']==1]
foward_center = new_data[new_data['nba_position_COLLEGE_F-C']==1]
fowards = new_data[(new_data['nba_position_COLLEGE_F']==1)]
forward_guard = new_data[(new_data['nba_position_COLLEGE_F-G']==1)]
guard_forward = new_data[new_data['nba_position_COLLEGE_G-F']==1]
guards = new_data[new_data['nba_position_COLLEGE_G']==1]
loop_list = [guards,guard_forward,forward_guard,fowards,foward_center,centers]

fold_scores_guards = []
fold_scores_guard_forward = []
fold_scores_forward_guard = []
fold_scores_forwards = []
fold_scores_forward_center = []
fold_scores_centers = []
ct = 0
for i in range(30):
    ct += 1
    print(ct)
    temp_loop_list = loop_list.copy()
    for idx, position in enumerate(temp_loop_list):
        shuffled = shuffle_df(position)
        y3 = get_y(shuffled)
        dataframe = drop_y(shuffled)
        train_data2, test_data2, y_train2, y_test2 = train_test_split(dataframe, y3,random_state = int(100*random.random()))
        grad_boost1 = GradientBoostingClassifier(learning_rate=.01, n_estimators=5000)
        grad_boost1.fit(train_data2, y_train2)
        yhat3 = grad_boost1.predict(test_data2)
        try:
            auc = roc_auc_score(y_test2.values.flatten(),yhat3)
        except:
            continue
        #print(auc,idx)
        if idx == 0:
            fold_scores_guards.append(auc)
        elif idx == 1:
            fold_scores_guard_forward.append(auc)
        elif idx == 2:
            fold_scores_forward_guard.append(auc)
        elif idx == 3:
            fold_scores_forwards.append(auc)
        elif idx == 4:
            fold_scores_forward_center.append(auc)
        else:
            fold_scores_centers.append(auc)

mean_g = np.mean(fold_scores_guards)
mean_gf = np.mean(fold_scores_guard_forward)
mean_fg = np.mean(fold_scores_forward_guard)
mean_f = np.mean(fold_scores_forwards)
mean_fc = np.mean(fold_scores_forward_center)
mean_c = np.mean(fold_scores_centers)
print('AVG guards: ', mean_g)
print('AVG guard-fowards: ', mean_gf)
print('AVG forward-guards: ', mean_fg)
print('AVG forwards: ', mean_f)
print('AVG forward-centers: ', mean_fc)
print('AVG centers: ', mean_c)                                                                     
print('mean of means: ', np.mean([mean_g,mean_gf,mean_fg,mean_f,mean_fc,mean_c]))

1
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
2
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
3
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
4
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
5
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
6
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
7
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
8
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
9
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
10
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
11
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
12
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
13
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
14
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
15
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
16
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
17
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
18
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
19
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
20
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
21
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
22
idx  0
idx  1
idx  2
idx  3
idx  4
idx  5
23
idx  0
idx  1
id

In [87]:
y_train3

,career_per
Ryan Anderson,16.5
Chris Jefferies,7.2
Walter Berry,16.6
Nick Van Exel,15.8
Evan Eschmeyer,10.6
...,...
Rashad McCants,12.9
Elfrid Payton,15.2
Isaiah Thomas,20.3
Fennis Dembo,7.8


In [91]:
y_train3

,career_per
Jeff Teague,17.1
Wesley Person,14.1
Luther Wright,-0.4
Danny Green,13.2
James Young,7.4
...,...
Johnny OBryant,8
Brandon Ingram,11.7
Steven Hunter,13
Kyle Weaver,10.3


In [103]:
scores_list_per = []
cnt1 =0 
for i in range(1):
    df = data.copy()
    df['career_per'] = career_per.values
    shuffled1 = shuffle_df(df)
    y3 = get_y_per(shuffled1)
    print(type(df['career_per'][0]))
    print(y3)
    df = drop_y_per(shuffled1)
    train_data3, test_data3, y_train3, y_test3 = train_test_split(df, y3,random_state = int(100*random.random()))
    print(type(train_data3))
    print(type(test_data3))
    print(type(y_train3))
    print(type(y_test3))
    #print(train_data3.values.shape)
    #print(y_train3.values.flatten())
    model12 = GradientBoostingRegressor(learning_rate=.01, n_estimators=50)
    model12.fit(train_data3, y_train3)
    yhat3 = model12.predict(test_data3)
    print(yhat3)
    break
    score = train_score_(y_test3)
    print(score)
    ct += 1
    print(ct)
    scores_list_per.append(auc)

<class 'numpy.float64'>
                  career_per
Rod Strickland            18
Brandon Armstrong        4.9
Antonio Burks            7.2
Jeff McInnis            12.6
Aaron McKie             12.8
...                      ...
Luis Flores             10.1
Joel Bolomboy           16.4
Dwayne McClain           8.3
Alton Ford               9.4
Scott Burrell           13.5

[1367 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan 

In [100]:
train_data3.columns

Index(['assist_percentage_COLLEGE', 'assists_COLLEGE',
       'block_percentage_COLLEGE', 'blocks_COLLEGE', 'box_plus_minus_COLLEGE',
       'defensive_box_plus_minus_COLLEGE',
       'defensive_rebound_percentage_COLLEGE', 'defensive_rebounds_COLLEGE',
       'defensive_win_shares_COLLEGE',
       'effective_field_goal_percentage_COLLEGE',
       ...
       'turnover_percentage_NBA is missing', 'turnovers_NBA is missing',
       'two_point_attempts_NBA is missing',
       'two_point_percentage_NBA is missing', 'two_pointers_NBA is missing',
       'two_pointers_assisted_percentage_NBA is missing',
       'usage_percentage_NBA is missing',
       'value_over_replacement_player_NBA is missing',
       'win_shares_NBA is missing',
       'win_shares_per_48_minutes_NBA is missing'],
      dtype='object', length=510)